In [1]:
#Imports:

#LO ÚNICO QUE HE CAMBIADO ES EL MODELO. Va ganando modelo 6 (hist random forest)

import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pylab as plt



In [8]:
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from collections import Counter
from sklearn.datasets import make_classification

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.preprocessing import LabelEncoder

from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression



# Cargar el modelo guardado
#model = joblib.load('./data/model3.sav')
model = joblib.load('./data/model6.sav')


#from imblearn.over_sampling import SMOTE



In [9]:
df=pd.read_csv('./data/diamonds_test.csv', index_col=0)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,city
id,,,,,,,,,,
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam


In [10]:
df.describe()
df.info()
cols = ['city','cut', 'color', 'clarity']

cat_list = []
for col in cols:
    cat = df[col].unique()
    cat_num = len(cat)
    cat_dict = {"categorical_variable":col,
                "number_of_possible_values":cat_num,
                "values":cat}
    cat_list.append(cat_dict)
    
categories = pd.DataFrame(cat_list).sort_values(by="number_of_possible_values",
                                                ascending=False).reset_index(drop=True)
categories

<class 'pandas.core.frame.DataFrame'>
Index: 13485 entries, 0 to 13484
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    13485 non-null  float64
 1   cut      13485 non-null  object 
 2   color    13485 non-null  object 
 3   clarity  13485 non-null  object 
 4   depth    13485 non-null  float64
 5   table    13485 non-null  float64
 6   x        13485 non-null  float64
 7   y        13485 non-null  float64
 8   z        13485 non-null  float64
 9   city     13485 non-null  object 
dtypes: float64(6), object(4)
memory usage: 1.1+ MB


,categorical_variable,number_of_possible_values,values
0,city,13,"[Amsterdam, Surat, Kimberly, Paris, Tel Aviv, ..."
1,clarity,8,"[SI1, VS1, VS2, VVS1, SI2, VVS2, IF, I1]"
2,color,7,"[F, J, H, D, I, G, E]"
3,cut,5,"[Very Good, Ideal, Premium, Good, Fair]"


In [11]:
#<one hot encoding:

df_one_hot_encoding = pd.get_dummies(df, 
                                          columns=cols, 
                                          drop_first=True)
df_one_hot_encoding.tail()


,carat,depth,table,x,y,z,city_Antwerp,city_Dubai,city_Kimberly,city_Las Vegas,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
id,,,,,,,,,,,,,,,,,,,,,
13480,0.57,61.9,56.0,5.35,5.32,3.30,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
13481,0.71,62.2,55.0,5.71,5.73,3.56,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
13482,0.70,61.6,55.0,5.75,5.71,3.53,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
13483,0.70,58.8,57.0,5.85,5.89,3.45,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
13484,0.40,62.4,55.0,4.70,4.73,2.94,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True


In [22]:
#label encoding
# Inicializar LabelEncoder
label_encoders = {}
categorical_cols = ['city', 'cut', 'color', 'clarity']

# Aplicar LabelEncoder a cada columna categórica
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

# Mostrar los datos con columnas codificadas
print("\nDatos con columnas codificadas:")
print(df)



Datos con columnas codificadas:
       carat  cut  color  clarity  depth  table     x     y     z  city
id                                                                     
0       0.79    4      2        2   62.7   60.0  5.82  5.89  3.67     0
1       1.20    2      6        4   61.0   57.0  6.81  6.89  4.18    10
2       1.57    3      4        2   62.2   61.0  7.38  7.32  4.57     3
3       0.90    4      2        2   63.8   54.0  6.09  6.13  3.90     3
4       0.50    4      2        4   62.9   58.0  5.05  5.09  3.19     0
...      ...  ...    ...      ...    ...    ...   ...   ...   ...   ...
13480   0.57    2      1        2   61.9   56.0  5.35  5.32  3.30     0
13481   0.71    2      5        5   62.2   55.0  5.71  5.73  3.56     8
13482   0.70    2      2        4   61.6   55.0  5.75  5.71  3.53    11
13483   0.70    4      2        3   58.8   57.0  5.85  5.89  3.45    10
13484   0.40    2      5        7   62.4   55.0  4.70  4.73  2.94     8

[13485 rows x 10 columns]


In [12]:
X_test = df_one_hot_encoding

In [13]:
predictions = model.predict(X_test)

In [14]:
predictions

array([2857.72143926, 5423.14335028, 9666.63032787, ..., 3068.34504253,
       2162.40999872,  788.69482877])

In [15]:
final=pd.DataFrame(predictions,columns=['price'])

In [16]:
final.reset_index(drop=False, inplace=True)

In [17]:
final.columns

Index(['index', 'price'], dtype='object')

In [18]:
final = final.rename(columns = {'index': 'id'})

In [19]:
final

,id,price
0,0,2857.721439
1,1,5423.143350
2,2,9666.630328
3,3,4012.402470
4,4,1672.175706
...,...,...
13480,13480,1624.153674
13481,13481,2360.942063
13482,13482,3068.345043
13483,13483,2162.409999


In [ ]:
#final=final.reset_index()

In [20]:
final.columns

Index(['id', 'price'], dtype='object')

In [24]:
submision=pd.read_csv('./data/sample_submission.csv',)

In [25]:
submision.head()

,id,price
0,0,685.639121
1,1,642.445785
2,2,746.907960
3,3,60.737590
4,4,650.451526


In [21]:
final.to_csv('./data/final6_.csv', index = False)

In [27]:
final

,id,price
0,0,2857.721439
1,1,5423.143350
2,2,9666.630328
3,3,4012.402470
4,4,1672.175706
...,...,...
13480,13480,1624.153674
13481,13481,2360.942063
13482,13482,3068.345043
13483,13483,2162.409999
